In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import texto as t
import math
import pandas as pd
import csv

In [2]:
import csv
training_data = []

with open('base_treino_secretaria.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=';')
    for row in readCSV:
        training_data.append({"class": row[0], "sentence": row[1]})

In [3]:
training_data

[{'class': 'saudacao', 'sentence': ' oi'},
 {'class': 'saudacao', 'sentence': ' bom dia, tudo bem?'},
 {'class': 'saudacao', 'sentence': ' olá tudo bem?'},
 {'class': 'saudacao', 'sentence': ' boa tarde'},
 {'class': 'despedida', 'sentence': ' tchau'},
 {'class': 'despedida', 'sentence': ' obrigada, até breve'},
 {'class': 'despedida', 'sentence': ' valeu, até mais'},
 {'class': 'despedida', 'sentence': ' ok, obrigado'},
 {'class': 'localizacao', 'sentence': ' aonde fica a secretaria?'},
 {'class': 'localizacao', 'sentence': ' qual a localizacao da secretaria?'},
 {'class': 'localizacao', 'sentence': ' qual o endereço da secretaria?'},
 {'class': 'localizacao', 'sentence': ' a secretaria fica aonde?'},
 {'class': 'horario',
  'sentence': ' qual o horario de funcionamento da secretaria?'},
 {'class': 'horario', 'sentence': ' até que horas a secretaria fica aberta?'},
 {'class': 'horario', 'sentence': ' qu horas a secretaria abre?'},
 {'class': 'horario', 'sentence': ' que horas a secret

In [5]:
# Selecionando um stemmer para PT
stemmer = SnowballStemmer("portuguese")
stopWords = list(set(stopwords.words('portuguese')))

#Crio uma lista das classes de intenções
classes = list()
classes = list(set([classe['class'] for classe in training_data]))

In [6]:
classes

['saudacao', 'despedida', 'horario', 'contato', 'localizacao']

In [14]:
#Probabilidades de C
#Neste exemplo são todas iguais pois distribui igualmente o numero de sentencas
    
prob_c = {}
for classe in classes:
        prob_c[classe] = 0
    
for data in training_data:
    for classe in classes:
        if data['class'] == classe:
            prob_c[classe] += 1
for classe in classes:
    prob_c[classe] = prob_c[classe]/len(training_data)

In [15]:
prob_c

{'contato': 0.2,
 'despedida': 0.2,
 'horario': 0.2,
 'localizacao': 0.2,
 'saudacao': 0.2}

In [63]:
import pdb

x = ''

dicionario_1 = dict()
for sentenca in training_data:
    # tokenizar cada sentenca
    
    if x != sentenca['class']:
        x = sentenca['class']
        dicionario_2 = dict()
     
    for word in nltk.word_tokenize(sentenca['sentence']):
        
        if word not in (stopWords + ['.',',','?']):
            # eliminando as terminacoes das palavras
            stemmed_word = stemmer.stem(word.lower())
              
            if stemmed_word not in dicionario_2:                    
                dicionario_2[stemmed_word] = 1            
            else:
                dicionario_2[stemmed_word] += 1
        #pdb.set_trace()
        dicionario_1[x] = dicionario_2
            
dicionario_1

{'contato': {'contat': 1,
  'e-mail': 1,
  'form': 1,
  'gost': 1,
  'numer': 1,
  'secret': 4,
  'telefon': 1},
 'despedida': {'brev': 1, 'obrig': 2, 'ok': 1, 'tchau': 1, 'val': 1},
 'horario': {'abert': 1,
  'abre': 1,
  'fech': 1,
  'fic': 1,
  'funcion': 1,
  'hor': 3,
  'horari': 1,
  'qu': 1,
  'secret': 4},
 'localizacao': {'aond': 2,
  'enderec': 1,
  'fic': 2,
  'localizaca': 1,
  'secret': 4},
 'saudacao': {'bem': 2,
  'boa': 1,
  'bom': 1,
  'dia': 1,
  'oi': 1,
  'olá': 1,
  'tard': 1,
  'tud': 2}}

In [70]:
# Vamos aplicar uma tecnica de suavizacao para tornar o modelo mais adequado

suavizacao = dicionario_1

for intencao in suavizacao:
    for word in suavizacao[intencao]:
         suavizacao[intencao][word] += 1
# primeiro adicionamos 1 pra cada palavra q ja existe em cada classe
suavizacao

{'contato': {'contat': 2,
  'e-mail': 2,
  'form': 2,
  'gost': 2,
  'numer': 2,
  'secret': 5,
  'telefon': 2},
 'despedida': {'brev': 2, 'obrig': 3, 'ok': 2, 'tchau': 2, 'val': 2},
 'horario': {'abert': 2,
  'abre': 2,
  'fech': 2,
  'fic': 2,
  'funcion': 2,
  'hor': 4,
  'horari': 2,
  'qu': 2,
  'secret': 5},
 'localizacao': {'aond': 3,
  'enderec': 2,
  'fic': 3,
  'localizaca': 2,
  'secret': 5},
 'saudacao': {'bem': 3,
  'boa': 2,
  'bom': 2,
  'dia': 2,
  'oi': 2,
  'olá': 2,
  'tard': 2,
  'tud': 3}}

In [71]:
for sentenca in training_data:
    # tokenizar cada sentenca
         
    for word in nltk.word_tokenize(sentenca['sentence']):
        if word not in (stopWords + ['.',',','?']):
        # eliminando as terminacoes das palavras
            stemmed_word = stemmer.stem(word.lower())
        
        for classe in classes:
            
            #Vamos acrescentar todo o vocabulario em todas as classes
        
            if stemmed_word not in suavizacao[classe]:
                suavizacao[classe][stemmed_word] = 1
              
suavizacao

{'contato': {'abert': 1,
  'abre': 1,
  'aond': 1,
  'bem': 1,
  'boa': 1,
  'bom': 1,
  'brev': 1,
  'contat': 2,
  'dia': 1,
  'e-mail': 2,
  'enderec': 1,
  'fech': 1,
  'fic': 1,
  'form': 2,
  'funcion': 1,
  'gost': 2,
  'hor': 1,
  'horari': 1,
  'localizaca': 1,
  'numer': 2,
  'obrig': 1,
  'oi': 1,
  'ok': 1,
  'olá': 1,
  'qu': 1,
  'secret': 5,
  'tard': 1,
  'tchau': 1,
  'telefon': 2,
  'tud': 1,
  'val': 1},
 'despedida': {'abert': 1,
  'abre': 1,
  'aond': 1,
  'bem': 1,
  'boa': 1,
  'bom': 1,
  'brev': 2,
  'contat': 1,
  'dia': 1,
  'e-mail': 1,
  'enderec': 1,
  'fech': 1,
  'fic': 1,
  'form': 1,
  'funcion': 1,
  'gost': 1,
  'hor': 1,
  'horari': 1,
  'localizaca': 1,
  'numer': 1,
  'obrig': 3,
  'oi': 1,
  'ok': 2,
  'olá': 1,
  'qu': 1,
  'secret': 1,
  'tard': 1,
  'tchau': 2,
  'telefon': 1,
  'tud': 1,
  'val': 2},
 'horario': {'abert': 2,
  'abre': 2,
  'aond': 1,
  'bem': 1,
  'boa': 1,
  'bom': 1,
  'brev': 1,
  'contat': 1,
  'dia': 1,
  'e-mail': 1,
  

In [89]:
# Calcular score de uma sentenca nova

def naive_bayes(sentenca, show_details=True):
    import math
    probabilidades = dict()
    for intencao in classes:
        
        score = 0.0
        somatorio = 0

        # tokenize each word in our new sentence
        for word in nltk.word_tokenize(sentenca):

            # verificar se a palavra está em alguma classe
            stemmed = stemmer.stem(word.lower())
            if stemmed in suavizacao[intencao]:
                                
                # p(c)*p(atr/c)

                prob_atr = suavizacao[intencao][stemmed]/sum(suavizacao[intencao].values())
                somatorio = somatorio + math.log10(prob_atr)
        
        probabilidades[intencao] = math.log10(prob_c[intencao]) + somatorio
        
        from collections import OrderedDict
        sorted_by_value = OrderedDict(sorted(probabilidades.items(), 
                                  key=lambda kv: kv[1], reverse=True))
                                    
            
    return sorted_by_value

In [90]:
naive_bayes("oi qual o horario da secretaria?")

OrderedDict([('horario', -4.65860754566205),
             ('contato', -4.838351570159206),
             ('localizacao', -4.838351570159206),
             ('saudacao', -5.236291578831244),
             ('despedida', -5.403575176537004)])

In [91]:
naive_bayes("bom dia aonde fica a secretaria?")

OrderedDict([('localizacao', -7.109676774159352),
             ('horario', -7.965032573212738),
             ('contato', -8.063919283598677),
             ('saudacao', -8.160829296606734),
             ('despedida', -8.539978624670994)])

In [92]:
naive_bayes("ok obrigada")

OrderedDict([('despedida', -3.057222202086365),
             ('saudacao', -3.9245377177754897),
             ('contato', -3.9245377177754897),
             ('localizacao', -3.9245377177754897),
             ('horario', -4.005395031886707)])

In [93]:
naive_bayes("bom dia, por favor, poderia me informar o horario de funcionamento da secretaria?")

OrderedDict([('horario', -7.664002577548756),
             ('contato', -8.063919283598677),
             ('localizacao', -8.063919283598677),
             ('saudacao', -8.160829296606734),
             ('despedida', -8.539978624670994)])

In [94]:
naive_bayes("bom dia")

OrderedDict([('saudacao', -3.322477726447527),
             ('despedida', -3.8353734524700087),
             ('contato', -3.9245377177754897),
             ('localizacao', -3.9245377177754897),
             ('horario', -4.005395031886707)])

In [95]:
naive_bayes("poderia me informar o e-mail da secretaria?")

OrderedDict([('contato', -2.9245377177754897),
             ('localizacao', -3.225567713439471),
             ('horario', -3.3064250275506875),
             ('despedida', -3.8353734524700087),
             ('saudacao', -3.9245377177754897)])

In [97]:
list(naive_bayes("poderia me informar o e-mail da secretaria?").keys())[0]

'contato'